<a href="https://colab.research.google.com/github/Vasilis-Kyriakopoulos/Finetune-LLM-/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
import pandas as pd


ds = load_dataset("maxscha/commitbench")
main_path = "/content/drive/MyDrive/FinetuneLLM_Ergasia/"

cols = ["diff", "message"]


ds["train"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_train.csv", index=False)


ds["validation"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_validation.csv",index=False)


ds["test"].select_columns(cols) \
    .to_pandas() \
    .to_csv(main_path+"commitbench_test.csv", index=False)


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

In [1]:
from google.colab import drive
import pandas as pd


drive.mount('/content/drive')
main_path = "/content/drive/MyDrive/FinetuneLLM_Ergasia/"
train_df = pd.read_csv(main_path + "commitbench_train_1pct.csv")
val_df = pd.read_csv(main_path + "commitbench_validation_1pct.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

input_csv = main_path + "commitbench_validation.csv"
output_csv = main_path + "commitbench_validation_1pct.csv"

chunk_size = 100_000
keep_frac = 0.01
first = True

for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    sampled = chunk.sample(frac=keep_frac, random_state=42)

    sampled.to_csv(
        output_csv,
        mode="a",
        index=False,
        header=first
    )
    first = False


KeyboardInterrupt: 

In [3]:
train_df.head()

,diff,message
0,diff --git a/lib/is_translatable.rb b/lib/is_t...,Adding some failing specs to get the translati...
1,diff --git a/src/Event.php b/src/Event.php\nin...,Added getDescription function to Event
2,diff --git a/templates/module/modularity-mod-s...,Dont initialize slider if column count is same...
3,diff --git a/src/FieldHandlers/BaseFileFieldHa...,letting the user open a file in the new tab (t...
4,diff --git a/bids/layout/tests/test_layout.py ...,TEST: Test upward propagation of dynamic gette...


In [4]:
val_df.head()

,diff,message
0,diff --git a/lib/dicom/anonymizer.rb b/lib/dic...,Added Anonymizer#to_anonymizer
1,diff --git a/comments-bundle/contao/Comments.p...,[Comments] Do not allow insert tags in comment...
2,diff --git a/src/MetaBox.php b/src/MetaBox.php...,Use specific types annotations for arrays in d...
3,diff --git a/header.js b/header.js\nindex <HAS...,"Fix lint, rlp isn't used"
4,diff --git a/taskw/warrior.py b/taskw/warrior....,"Merge the ""waiting"" list back into the ""pendin..."


In [5]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


def token_len(t: str) -> int:
    return len(tokenizer.encode(t, add_special_tokens=True))


rng = np.random.default_rng(42)
texts = train_df.apply(format_input, axis=1).tolist()
sample_size = min(50_000, len(texts))
sample_texts = rng.choice(texts, size=sample_size, replace=False)

lengths = np.array([token_len(t) for t in sample_texts], dtype=np.int32)

p = [50, 75, 90, 95, 97, 98, 99, 99.5]
qs = np.percentile(lengths, p)

for perc, q in zip(p, qs):
    print(f"{perc:>5}%  -> {int(q)} tokens")

# Example choice:
max_len = int(np.percentile(lengths, 95))
print("Suggested max_len:", max_len)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NameError: name 'format_input' is not defined

In [6]:
import torch
from torch.utils.data import Dataset
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))
def format_input(entry):

    input_text = f"{entry['diff']}\nCommit Message:\n{entry['message']}"

    return input_text
print(train_df.iloc[0])
print(format_input(train_df.iloc[0]))


class CodeDiffMessageDataset(Dataset):
    def __init__(self, data, tokenizer,max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 1024


    def __getitem__(self, index):
        self.encoded_text = None
        entry = self.data.iloc[index]
        code_diff_plus_message = format_input(entry)
        self.encoded_text = tokenizer.encode(code_diff_plus_message,max_length=self.max_length,Truncation=True)
        return self.encoded_text

    def __len__(self):
        return int(len(self.data)/10)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)

def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index


        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)

[50256]
diff       diff --git a/lib/is_translatable.rb b/lib/is_t...
message    Adding some failing specs to get the translati...
Name: 0, dtype: object
diff --git a/lib/is_translatable.rb b/lib/is_translatable.rb
index <HASH>..<HASH> 100644
--- a/lib/is_translatable.rb
+++ b/lib/is_translatable.rb
@@ -13,6 +13,9 @@ module IsTranslatable
 
     def set_translation(kind, t, locale_override=nil)
     end
+
+	def get_translation(kind, locale_override=nil)
+	end
   end
 end
 
diff --git a/spec/lib/is_translatable_spec.rb b/spec/lib/is_translatable_spec.rb
index <HASH>..<HASH> 100644
--- a/spec/lib/is_translatable_spec.rb
+++ b/spec/lib/is_translatable_spec.rb
@@ -24,7 +24,20 @@ describe IsTranslatable do
       context 'with translated title' do
         before {@article.set_translation(:title, @titles[:es])}
         it {should be_valid}
-        it "should check that it's actually translated"
+
+        it {subject.get_translation(:title).should == @titles[:es]}
+
+		context 'loaded from

In [ ]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_fn(batch))

In [7]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 4

torch.manual_seed(123)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)
val_dataset = CodeDiffMessageDataset(val_df, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle= False,
    drop_last=True,
    num_workers=num_workers
)

In [ ]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

In [ ]:

print(inputs[5])

In [ ]:

print(targets[5])

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False
model.config.pad_token_id = model.config.eos_token_id
print(tokenizer.vocab_size)
model.config.pad_token_id





50257


50256

In [9]:
print(model.config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": false,
  "vocab_size": 50257
}



In [10]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import math


def calculate_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=-1) #(batch_size, sequence_length, vocab_size)
    correct_predictions = (predictions == labels).float() # (batch_size, sequence_length)
    return correct_predictions.mean().item()

def generate(model, tokenizer, device):
    example_diff = "--- a/file.py\n+++ b/file.py\n- print('hi')\n+ print('hello')"
    prompt = f"Diff:\n{example_diff}\n\nCommit Message:\n"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,
        padding=True,
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"\n[Generated]: {generated_text}")

# --- 2. Training Step ---
def train_one_epoch(model, dataloader, optimizer, device, progress_bar, tokenizer):
    model.train()
    total_loss = 0
    total_acc = 0

    for i, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # Forward pass
        logits = model(x).logits
        loss = nn.CrossEntropyLoss()(
            logits.view(-1, logits.size(-1)),
            y.view(-1)
        )
        acc = calculate_accuracy(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += acc

        progress_bar.update(1)
        progress_bar.set_postfix({"train_loss": f"{loss.item():.4f}","acc": f"{acc:.4f}"})


        if i > 0 and i % 100 == 0:
            generate(model, tokenizer, device)

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc

# --- 3. Evaluation Step ---
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_acc=0
    with torch.no_grad():

        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x).logits
            loss = nn.CrossEntropyLoss()(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_loss += loss.item()
            total_acc  += calculate_accuracy(logits, y)


    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc


# --- 4. Main Loop ---
def train(model, train_loader, val_loader, optimizer, device, tokenizer):
    best_val_loss = float("inf")
    patience = 2
    bad_epochs = 0
    epochs = 2

    for epoch in range(epochs):

        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        # Train
        train_loss,train_acc = train_one_epoch(model, train_loader, optimizer, device, progress_bar, tokenizer)

        # Evaluate
        val_loss,val_acc = evaluate(model, val_loader, device)
        try:
            ppl = math.exp(val_loss)
        except OverflowError:
            ppl = float('inf')

        progress_bar.set_postfix({
            "train_loss": f"{train_loss:.4f}",
            "train_accuracy": f"{train_acc:.4f}",
            "val_loss": f"{val_loss:.4f}",
            "val_accuracy": f"{val_acc:.4f}",
            "PPL": f"{ppl:.2f}"
        })
        progress_bar.refresh()
        progress_bar.close() # Close bar to print new line

        # Logging
        print(f"Summary Epoch {epoch+1}: Train Loss: {train_loss:.4f}| Train Accuracy: {train_acc:.4f} \
        | Val Loss: {val_loss:.4f}|Val Accuracy: {val_acc:.4f})| PPL: {ppl:.4f}")

        # Save checkpoints
        torch.save(model.state_dict(), f"{main_path}/gpt2_epoch{epoch+1}.pt")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            bad_epochs = 0
            torch.save(model.state_dict(), f"{main_path}/best_model.pt")
            print(">>> New best model saved!")
        else:
            bad_epochs += 1
            print(f">>> No improvement (bad epochs: {bad_epochs})")

        if bad_epochs >= patience:
            print("EARLY STOPPING TRIGGERED.")
            break

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
train(model, train_loader, val_loader, optimizer, device, tokenizer)

Epoch 1/2:   0%|          | 0/316 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.



[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fixes #<I> issue.

[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
File: <URL>/files>

[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fix bug #<I>
Summary Epoch 1: Train Loss: 1.7154 | Val Loss: 1.5253| PPL: 4.60
>>> New best model saved!


Epoch 2/2:   0%|          | 0/316 [00:00<?, ?it/s]


[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fixes #<I>

[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fix #<I>

[Generated]: Diff:
--- a/file.py
+++ b/file.py
- print('hi')
+ print('hello')

Commit Message:
Fix bug in Python <I>
Summary Epoch 2: Train Loss: 1.3676 | Val Loss: 1.5076| PPL: 4.52
>>> New best model saved!


In [11]:
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Setup the App
app = FastAPI(title="Git Diff to Commit Message API")

# 2. Define Request Schema
# This ensures users send valid JSON with a "diff" field
class DiffRequest(BaseModel):
    diff: str
    max_tokens: int = 50  # Default value, but user can change it

# 3. Global Variables for Model (Loaded on Startup)
model = None
tokenizer = None
device = None

@app.on_event("startup")
def load_model():
    global model, tokenizer, device

    # Configuration
    MODEL_PATH = "best_model.pt" # Or your specific checkpoint path
    BASE_MODEL_NAME = "gpt2"     # Needed for the tokenizer configuration

    print("Loading model and tokenizer...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Tokenizer (Use the base gpt2 tokenizer)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    # Load Model Structure & Weights
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)
    model.resize_token_embeddings(len(tokenizer)) # Important if you resized during training

    # Load your fine-tuned weights
    # map_location ensures it loads even if you move from GPU -> CPU
    state_dict = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict)

    model.to(device)
    model.eval()
    print(f"Model loaded successfully on {device}!")

# 4. Generation Endpoint
@app.post("/generate")
async def generate_commit_message(request: DiffRequest):
    if not model:
        raise HTTPException(status_code=500, detail="Model not loaded")

    try:
        # Format the input exactly how we trained it
        prompt = f"Diff:\n{request.diff}\n\nCommit Message:\n"

        # Tokenize
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True,
            padding=True
        ).to(device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=request.max_tokens,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Post-processing: Extract only the commit message part
        # We split by our known separator and take the last part
        answer = full_text.split("Commit Message:\n")[-1].strip()

        return {
            "generated_message": answer,
            "full_text": full_text
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# 5. Health Check
@app.get("/health")
def health_check():
    return {"status": "ok", "device": str(device)}

/tmp/ipython-input-463976673.py:20: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [20]:
!ls

best_model.pt			 commitbench_validation.csv
commitbench_test.csv		 GenerateApi.py
commitbench_train_1pct.csv	 gpt2_epoch1.pt
commitbench_train.csv		 gpt2_epoch2.pt
commitbench_validation_1pct.csv  gpt2_first_training.pt


In [22]:
!uvicorn GenerateApi:app --reload

INFO:     Will watch for changes in these directories: ['/content/drive/MyDrive/FinetuneLLM_Ergasia']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [21363] using WatchFiles
INFO:     Started server process [21369]
INFO:     Waiting for application startup.
Loading model and tokenizer...
2026-01-18 16:06:32.332422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768752392.363738   21369 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768752392.374090   21369 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768752392.400656   21369 computation_placer.cc:177] computation placer al

In [27]:
import requests

url = "http://127.0.0.1:8000/generate"
data = {"diff": "--- a/file.py\n+++ b/file.py\n- print('error')\n+ print('fixed')"}

response = requests.post(url, json=data)
print(response.json()['generated_message'])

fix(file) fix error handling
